A symbolic representation of differenital drive vehicle dynamics with x-axis and y-axis offset reference point (reference to the driveshaft midpoint)

Equations are from the paper written by Masayoshi Wada etal.

https://www.jstage.jst.go.jp/article/jrsj1983/18/8/18_8_1166/_pdf

This notebook is based on [Yosuke Matsusaka repo](https://github.com/devrt/offset-diff-drive-controller/tree/master)

# Preparation

We use sympy to transform the equation:

In [85]:
#!sudo pip install -U sympy

In [86]:
from sympy import *

In [87]:
init_printing()

# Symbols

Following symbols are used to describe internal state of the vehicle:

In [88]:
(r, s_x, s_y, W) = symbols(r"r s_x s_y W")

$r$ : radius of each wheels

$W$: wheel separation

$s_x$ : wheel offset in x-axis

$s_y$ : wheel offset in y-axis

Joint space parameters:

In [89]:
(wr, wl) = symbols('w_r w_l')
uv = Matrix([wr, wl])

$w_r$ : angular velocity of the right wheel joint

$w_l$ : angular velocity of the left wheel joint

Cartesian space parameters:

In [90]:
(vx, vy, omega) = symbols('v_x v_y \omega')
twist_v = Matrix([vx, omega])
v_des = Matrix([vx, vy])

$v_x$ : x axes velocity

$v_y$ : y axes velocity

$\omega_z$ : yaw speed

# Dynamics of diff drive vehicle

In [91]:
diff_drive = Matrix([
    [r/2, r/2],
    [r/W, -r/W]
])
Eq(xv, MatMul(diff_drive, Matrix(uv[0:2]), evaluate=False), evaluate=False)

           ⎡r   r ⎤     
           ⎢─   ─ ⎥     
⎡  vₓ  ⎤   ⎢2   2 ⎥ ⎡wᵣ⎤
⎢      ⎥ = ⎢      ⎥⋅⎢  ⎥
⎣\omega⎦   ⎢r  -r ⎥ ⎣wₗ⎦
           ⎢─  ───⎥     
           ⎣W   W ⎦     

x axes velocity $v_x$ and yaw velocity $\omega_z$ is generated from the combination of left and right wheel velocities.
We can natually understand that we cannot directly obtain y axes velocity from the mechanism.

# Dynamics of offset diff drive vehicle

Here, we apply offset to the diff drive vehicle described above.

We first, create offset transformation matrix:

In [92]:
offset = Matrix([
    [1, s_y],
    [0, s_x]
])
offset



⎡1  s_y⎤
⎢      ⎥
⎣0  sₓ ⎦

In [93]:
inv_offset = offset.inv()

inv_offset

⎡   -s_y ⎤
⎢1  ─────⎥
⎢     sₓ ⎥
⎢        ⎥
⎢    1   ⎥
⎢0   ──  ⎥
⎣    sₓ  ⎦

In [94]:

Eq(v_des, MatMul(offset, Matrix(twist_v[0:2]), evaluate=False), evaluate=False)

⎡vₓ ⎤   ⎡1  s_y⎤ ⎡  vₓ  ⎤
⎢   ⎥ = ⎢      ⎥⋅⎢      ⎥
⎣v_y⎦   ⎣0  sₓ ⎦ ⎣\omega⎦

In [95]:
Eq(twist_v, MatMul(inv_offset, Matrix(v_des[0:2]), evaluate=False), evaluate=False)

           ⎡   -s_y ⎤      
           ⎢1  ─────⎥      
⎡  vₓ  ⎤   ⎢     sₓ ⎥ ⎡vₓ ⎤
⎢      ⎥ = ⎢        ⎥⋅⎢   ⎥
⎣\omega⎦   ⎢    1   ⎥ ⎣v_y⎦
           ⎢0   ──  ⎥      
           ⎣    sₓ  ⎦      

Apply offset to the jacobian matrix:

In [96]:
offset_diff_drive = offset * diff_drive
Eq(offset_diff_drive, MatMul(offset, diff_drive, evaluate=False), evaluate=False)

⎡r   r⋅s_y  r   r⋅s_y⎤            ⎡r   r ⎤
⎢─ + ─────  ─ - ─────⎥            ⎢─   ─ ⎥
⎢2     W    2     W  ⎥   ⎡1  s_y⎤ ⎢2   2 ⎥
⎢                    ⎥ = ⎢      ⎥⋅⎢      ⎥
⎢  r⋅sₓ      -r⋅sₓ   ⎥   ⎣0  sₓ ⎦ ⎢r  -r ⎥
⎢  ────      ──────  ⎥            ⎢─  ───⎥
⎣   W          W     ⎦            ⎣W   W ⎦

Now, the equation will be as below:

In [97]:
Eq(v_des, MatMul(offset_diff_drive, Matrix(uv[0:2]), evaluate=False), evaluate=False)

        ⎡r   r⋅s_y  r   r⋅s_y⎤     
        ⎢─ + ─────  ─ - ─────⎥     
⎡vₓ ⎤   ⎢2     W    2     W  ⎥ ⎡wᵣ⎤
⎢   ⎥ = ⎢                    ⎥⋅⎢  ⎥
⎣v_y⎦   ⎢  r⋅sₓ      -r⋅sₓ   ⎥ ⎣wₗ⎦
        ⎢  ────      ──────  ⎥     
        ⎣   W          W     ⎦     

What is interesting here, is we get y axes velocity by just adding the offset!

Calculating the inverse matrix, we get:

In [98]:
inv_offset_diff_drive = Matrix(offset_diff_drive[0:2, :]).inv()
# inv_offset_diff_drive = Matrix(offset_diff_drive).pinv()

Eq(Matrix(uv[0:2]), MatMul(inv_offset_diff_drive, Matrix(xv[0:2]), evaluate=False), evaluate=False)

       ⎡1  W - 2⋅s_y ⎤         
       ⎢─  ───────── ⎥         
⎡wᵣ⎤   ⎢r    2⋅r⋅sₓ  ⎥ ⎡  vₓ  ⎤
⎢  ⎥ = ⎢             ⎥⋅⎢      ⎥
⎣wₗ⎦   ⎢1  -W - 2⋅s_y⎥ ⎣\omega⎦
       ⎢─  ──────────⎥         
       ⎣r    2⋅r⋅sₓ  ⎦         